In [2]:
## Run this cell once without comments to install packages, then comment them out.
#!pip install openai
#!pip install langchain
#!pip install tiktoken
#!pip install faiss-cpu

# Don't change

In [3]:
from langchain.chat_models import AzureChatOpenAI, ChatVertexAI
from langchain.embeddings import OpenAIEmbeddings, VertexAIEmbeddings
from langchain.schema import (
    AIMessage, 
    HumanMessage, 
    SystemMessage
)
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import os
os.environ["OPENAI_API_KEY"] = "be9bdecc8bf64e85bde69c04b2ad56f8"
os.environ["OPENAI_API_HOST"] = "https://tiaa-openai-azure-sweden.openai.azure.com/"
os.environ["OPENAI_API_EMBEDDING_KEY"] = "be9bdecc8bf64e85bde69c04b2ad56f8"
os.environ["OPENAI_API_EMBEDDING_HOST"] = "https://tiaa-openai-azure-sweden.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] =  "2023-07-01-preview"
import openai
openai.api_type='azure'
openai.api_base = os.environ["OPENAI_API_HOST"]
openai.api_key = os.environ["OPENAI_API_KEY"]
#model_name='tiaa-gpt-4-32k'
base_model_name="tiaa-gpt-4-32k"
chosen_llm = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0
    )
embedding_model_name = 'tiaa-text-embedding-ada-002'
chosen_embedding_model = OpenAIEmbeddings(
        deployment=embedding_model_name,
        openai_api_base=os.environ["OPENAI_API_EMBEDDING_HOST"],
        openai_api_version= os.environ["OPENAI_API_VERSION"],
        openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
        openai_api_type="azure",
        chunk_size=16
    )
finalAnswerTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0,
        verbose = True
    )
condensingTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0, 
        verbose = True
    )
memory = ConversationBufferMemory(
    output_key='answer',
    memory_key='chat_history',
    return_messages=True
    )



## Asking general questions

In [45]:
msg = HumanMessage(content="Where are capitals of France and England? Which one is a bigger city")
output=chosen_llm(messages=[msg])
print(output.content)

The capital of France is Paris and the capital of England is London. London is a bigger city in terms of both population and area.


## Defining a sample text

In [37]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
text2=f"""hello world!"""

## Asking questions from the raw text

In [38]:
msg = HumanMessage(content=f" What is the main  topic discussed in the text?  ```{text}```")
output=chosen_llm(messages=[msg])
print(output.content)
msg = HumanMessage(content=f" How many characters are there in the text?  ```{text2}```")
output=chosen_llm(messages=[msg])
print(output.content)

The main topic discussed in the text is the importance of providing clear and specific instructions to a model to guide it towards the desired output and reduce the chances of irrelevant or incorrect responses.
There are 12 characters in the text "hello world!".


## Embedding a text

In [39]:
def vectorstore(text):
    new_doc = Document(page_content=text, metadata=[])
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    chunked_docs = text_splitter.split_documents([new_doc])
    vectorstore = FAISS.from_documents(chunked_docs, chosen_embedding_model)
    return(vectorstore)

In [40]:
current_vector=vectorstore(text)

## Asking questions from the embedded text

In [41]:
qa = ConversationalRetrievalChain.from_llm(

    llm = finalAnswerTheQuestionModel,

    retriever = current_vector.as_retriever(),

    condense_question_llm = condensingTheQuestionModel,

)

source_with_qa = ConversationalRetrievalChain.from_llm(

    llm = finalAnswerTheQuestionModel,

    retriever = current_vector.as_retriever(

                                         search_kwargs={

                                            "k": 8 #num_nearest_neighbors.value,

                                            }),

    condense_question_llm = condensingTheQuestionModel,

    memory=memory,

    return_source_documents=True,

    verbose=False

)

In [42]:
chat_history = []
query = f" What is the main  topic discussed in this document?"
result = qa({"question": query, "chat_history": chat_history})
result['answer']


'The main topic discussed in this document is the importance of providing clear and specific instructions to a model to guide it towards the desired output and reduce the chances of receiving irrelevant or incorrect responses. It also emphasizes that longer prompts can often provide more clarity and context for the model.'